In [75]:
# Do not edit this cell

# course: 3654
# a: Project 1
# d: VT

# S&P 500 Companies Pre/Post Covid

### Team: Rich Homies (Group 29)

**Name:**  Thejus Poruthikode Unnivelan
**PID:**  thejuspu

**Name:**  Andrew Visocan
**PID:**  avisocan
    
We have neither given nor received unauthorized assistance on this assignment. See the course sylabus for details on the Honor Code policy. In particular, sharing lines of solution code is prohibited.

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import datetime
import plotly.graph_objs as go                            # pip install plotly in Anaconda Prompt

companyFile = "S&P500_Companies.csv"
companies = pd.read_csv(companyFile)
companies.columns

Index(['Ticker', 'Company', 'Sector', 'Industry'], dtype='object')

## Objective

To organize the companies on the S&P 500 into groups that flourished or struggled in the times of Covid-19, based on OHLCV data (Open, High, Low, Close, Volume). companyFile is a file containing details on the 509 companies on the S&P500, including ticker symbol, company name, their sector, and their industry (individual industries make up sectors).

In [43]:
OHLCV = {}
for i in companies['Ticker']:
    temp = pd.read_csv("./DATA/" + i + ".csv")
    dates = temp['date']
    dateTimes = []
    for j in dates:
        dateTimes.append(datetime.strptime(j, '%Y-%m-%d'))
    temp['date'] = dateTimes
    temp.set_index('date', inplace=True)
    OHLCV[i] = temp

In [45]:
OHLCV['A'].columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'date'], dtype='object')

### Normalization
This method normalizes the data. SKLearn's Preprocessing's MinMaxScaler allows you to scale s ndarray of values between a desired range (we used (0, 1) and that is the default) by using the max and min value within the ndarray.

We wanted all the output to open at the same number so we divided by the initial opening value, after adding 1 to the numerator and denomenator. The division makes values less than the opening value between 0 and 1 and the values greater than the opening value large. So we found the natural log of the division and to avoid accidentally finding the natural log of zero, we added 1 to the numberator and denominator in the division step.

In [59]:
# Pre-req: data = OHLCV[ticker].drop('date', axis=1).drop('Volume', axis=1)
# and/or
# Pre-req: data = OHLCV[ticker].Volume
def normalize(data):
    normaliser = preprocessing.MinMaxScaler()
    temp = data.values
    temp = normaliser.fit_transform(temp)
    temp = (temp + .001) / (temp[0][0] + .001)
    temp = np.log(temp)
    return temp

def normOHLCV(data):
    temp = normalize(data.drop('Volume', axis=1).drop('date', axis=1))
    temp = pd.DataFrame(data=temp, columns=['Open', 'High', 'Low', 'Close'])
    temp['Volume'] = data['Volume']
    temp['date'] = data['date']
    return temp


### Normalization Pre-, Post-Covid and Crash (Pre Crash Post (PCP))

normPCP is a dictionary, key'd by the Ticker symbol, containing the set of three OHLCV data within the time frames and normalizes OHLC and Volume within those time frames.

#### Time Frames:
Pre-Covid: [start, preCrashHigh]

Crash: [preGrashHigh, postCrashLow]

Post-Covid: [postCrashLow, end]

In [60]:
crash = datetime(2020, 2, 20)
start = datetime(2019, 8, 20)
end = datetime(2020, 8, 20)
preCrashHigh = datetime(2020, 2, 20)
postCrashLow = datetime(2020, 3, 23)

In [67]:
normPCP = {}
for i in companies['Ticker']:
    data = OHLCV[i]
    pre = data.loc[:preCrashHigh, :]
    crash = OHLCV[i].loc[preCrashHigh:postCrashLow, :]
    post = OHLCV[i].loc[postCrashLow:, :]
    if (pre.shape[0] != 0):
        normPCP[i] = (normOHLCV(pre), normOHLCV(crash), normOHLCV(post))

503


In [70]:
print('Removed' + str(len(OHLCV.keys()) - len(normPCP.keys())) 'Companies for not containing long enough information' )

SyntaxError: invalid syntax (<ipython-input-70-9dc044ff03b8>, line 1)

In [93]:
for i in set(companies['Sector']):
    print('\t\t\t', '\033[1m' + i + '\033[0m')
    sector = companies[companies['Sector'] == i]
    for j in set(sector['Industry']):
        print(j)
        industry = sector[sector['Industry'] == j]
        for k in industry.index:
            print(industry.loc[k, 'Ticker'], ':\t', industry.loc[k, 'Company'])
        print()
    print()

			 Basic Materials
Building Materials
MLM :	 Martin Marietta Materials, Inc.
VMC :	 Vulcan Materials Company

Chemicals
APD :	 Air Products and Chemicals, Inc.
CE :	 Celanese Corporation
DD :	 DuPont de Nemours, Inc.
DOW :	 Dow Inc.
EMN :	 Eastman Chemical Company

Gold
NEM :	 Newmont Corporation

Copper
FCX :	 Freeport-McMoRan Inc.

Agricultural Inputs
CF :	 CF Industries Holdings, Inc.
CTVA :	 Corteva, Inc.
FMC :	 FMC Corporation
MOS :	 The Mosaic Company

Steel
NUE :	 Nucor Corporation

Specialty Chemicals
ALB :	 Albemarle Corporation
ECL :	 Ecolab Inc.
IFF :	 International Flavors & Fragrances Inc.
LIN :	 Linde plc
LYB :	 LyondellBasell Industries N.V.
PPG :	 PPG Industries, Inc.
SHW :	 The Sherwin-Williams Company


			 Energy
Oil & Gas E&P
APA :	 Apache Corporation
COG :	 Cabot Oil & Gas Corporation
COP :	 ConocoPhillips
CXO :	 Concho Resources Inc.
DVN :	 Devon Energy Corporation
EOG :	 EOG Resources, Inc.
FANG :	 Diamondback Energy, Inc.
HES :	 Hess Corporation
MRO :	 Marathon

UPS :	 United Parcel Service, Inc.

Specialty Business Services
CPRT :	 Copart, Inc.
CTAS :	 Cintas Corporation
GPN :	 Global Payments Inc.

Building Products & Equipment
CARR :	 Carrier Global Corporation
MAS :	 Masco Corporation

Railroads
CSX :	 CSX Corporation
KSU :	 Kansas City Southern
NSC :	 Norfolk Southern Corporation
UNP :	 Union Pacific Corporation
WAB :	 Wabtec Corporation

Consulting Services
EFX :	 Equifax Inc.
INFO :	 IHS Markit Ltd.
NLSN :	 Nielsen Holdings plc
VRSK :	 Verisk Analytics, Inc.

Aerospace & Defense
BA :	 The Boeing Company
GD :	 General Dynamics Corporation
HII :	 Huntington Ingalls Industries, Inc.
LHX :	 L3Harris Technologies, Inc.
LMT :	 Lockheed Martin Corporation
NOC :	 Northrop Grumman Corporation
RTX :	 Raytheon Technologies Corporation
TDG :	 TransDigm Group Incorporated
TXT :	 Textron Inc.

Business Equipment & Supplies
AVY :	 Avery Dennison Corporation


			 Consumer Cyclical
Footwear & Accessories
NKE :	 NIKE, Inc.

Personal Services
HRB :	 H&R 

## Companies by Sector & Industry

In [2]:
print(len(set(companies['Sector'])), 'Sectors Total')
for i in set(companies['Sector']):
    sector = companies[companies['Sector'] == i]
    print(len(set(sector['Industry'])), '\t', '\033[1m' + i + '\033[0m')
    sector = companies[companies['Sector'] == i]
    for j in set(sector['Industry']):
        print(j)
    print()

11 Sectors Total
18 	 Industrials
Airlines
Rental & Leasing Services
Trucking
Railroads
Building Products & Equipment
Consulting Services
Staffing & Employment Services
Industrial Distribution
Engineering & Construction
Specialty Industrial Machinery
Security & Protection Services
Tools & Accessories
Specialty Business Services
Waste Management
Integrated Freight & Logistics
Business Equipment & Supplies
Farm & Heavy Construction Machinery
Aerospace & Defense

5 	 Energy
Oil & Gas Integrated
Oil & Gas Equipment & Services
Oil & Gas Midstream
Oil & Gas Refining & Marketing
Oil & Gas E&P

6 	 Communication Services
Internet Content & Information
Telecom Services
Broadcasting
Entertainment
Electronic Gaming & Multimedia
Advertising Agencies

20 	 Consumer Cyclical
Auto Parts
Personal Services
Lodging
Restaurants
Travel Services
Auto & Truck Dealerships
Auto Manufacturers
Department Stores
Specialty Retail
Apparel Manufacturing
Internet Retail
Home Improvement Retail
Resorts & Casinos
Luxu

In [3]:
for i in set(companies['Sector']):
    print('\t\t\t', '\033[1m' + i + '\033[0m')
    sector = companies[companies['Sector'] == i]
    for j in set(sector['Industry']):
        print(j)
        industry = sector[sector['Industry'] == j]
        for k in industry.index:
            print(industry.loc[k, 'Ticker'], ':\t', industry.loc[k, 'Company'])
        print()
    print()

			 Industrials
Airlines
AAL :	 American Airlines Group Inc.
ALK :	 Alaska Air Group, Inc.
DAL :	 Delta Air Lines, Inc.
LUV :	 Southwest Airlines Co.
UAL :	 United Airlines Holdings, Inc.

Rental & Leasing Services
URI :	 United Rentals, Inc.

Trucking
ODFL :	 Old Dominion Freight Line, Inc.

Railroads
CSX :	 CSX Corporation
KSU :	 Kansas City Southern
NSC :	 Norfolk Southern Corporation
UNP :	 Union Pacific Corporation
WAB :	 Wabtec Corporation

Building Products & Equipment
CARR :	 Carrier Global Corporation
MAS :	 Masco Corporation

Consulting Services
EFX :	 Equifax Inc.
INFO :	 IHS Markit Ltd.
NLSN :	 Nielsen Holdings plc
VRSK :	 Verisk Analytics, Inc.

Staffing & Employment Services
ADP :	 Automatic Data Processing, Inc.
PAYX :	 Paychex, Inc.
RHI :	 Robert Half International Inc.

Industrial Distribution
FAST :	 Fastenal Company
GWW :	 W.W. Grainger, Inc.

Engineering & Construction
J :	 Jacobs Engineering Group Inc.
JCI :	 Johnson Controls International plc
PWR :	 Quanta Service

SIVB :	 SVB Financial Group
TFC :	 Truist Financial Corporation
USB :	 U.S. Bancorp
ZION :	 Zions Bancorporation, National Association

Banks - Diversified
BAC :	 Bank of America Corporation
C :	 Citigroup Inc.
JPM :	 JPMorgan Chase & Co.
WFC :	 Wells Fargo & Company

Insurance - Specialty
AIZ :	 Assurant, Inc.

Capital Markets
ETFC :	 E*TRADE Financial Corporation
GS :	 The Goldman Sachs Group, Inc.
MKTX :	 MarketAxess Holdings Inc.
MS :	 Morgan Stanley
RJF :	 Raymond James Financial, Inc.
SCHW :	 The Charles Schwab Corporation

Insurance Brokers
AJG :	 Arthur J. Gallagher & Co.
AON :	 Aon Plc
MMC :	 Marsh & McLennan Companies, Inc.
WLTW :	 Willis Towers Watson Public Limited Company


			 Basic Materials
Gold
NEM :	 Newmont Corporation

Building Materials
MLM :	 Martin Marietta Materials, Inc.
VMC :	 Vulcan Materials Company

Agricultural Inputs
CF :	 CF Industries Holdings, Inc.
CTVA :	 Corteva, Inc.
FMC :	 FMC Corporation
MOS :	 The Mosaic Company

Specialty Chemicals
ALB :	 Albema

## Interactive Candle-Stick Graph
The first thing we tried to do was try to find patterns by looking at the Candle Stick Graph, which ended up taking too much time and couldn't overlay multiple candle-stick graphs on top of each other so we decided to use a line graph for this analysis instead.

Instead if we want to view any single graph we can use the cell two cells down to create an interactive candle-stick graph.

In [9]:
def createCandleStick(data):
    plot=[go.Candlestick(x=data.date,
                open=data.Open,
                high=data.High,
                low=data.Low,
                close=data.Close)]
    figSignal = go.Figure(data=plot)
    figSignal.show()

In [65]:
ticker = 'CARR'
createCandleStick(OHLCV[ticker])